In [1]:
import sys
sys.path.append('../input/multilabelstraifier/')
from ml_stratifiers import MultilabelStratifiedKFold

import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
DIR = "../input/moa-ensemble-check/"
check_dae = pd.read_csv(DIR+"dae_oof.csv").drop("sig_id", axis=1)
check_mlp = pd.read_csv(DIR+"mlp_oof2.csv")
check_tabnet = pd.read_csv(DIR+"tab_oof2.csv")
check_svm = pd.read_csv(DIR+"svm_oof2.csv")

y = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv').drop("sig_id", axis=1)
train = pd.read_csv('../input/lish-moa/train_features.csv')
drug = pd.read_csv('../input/lish-moa/train_drug.csv')

#mlp = pd.read_csv(DIR+"mlp1840.csv")
#check_tabnet = pd.read_csv(DIR+"tab_oof2.csv")
#tab = pd.read_csv(DIR+"sub_tab1859.csv")
#xgb = pd.read_csv(DIR+"xgb1951.csv")
#check_xgb = pd.read_csv(DIR+"xgb_oof.csv")

In [2]:
cons_train_index = train[train.cp_type!="ctl_vehicle"].index

In [3]:
y = y.loc[y.index.isin(cons_train_index),:]

In [4]:
corr_df = y.corr().abs()
corr_df

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
5-alpha_reductase_inhibitor,1.000000,0.000798,0.000921,0.002602,0.003283,0.001608,0.001383,0.001845,0.000651,0.003107,...,0.000460,0.000940,0.001303,0.003365,0.001608,0.000460,0.002460,0.000959,0.001175,0.001030
11-beta-hsd1_inhibitor,0.000798,1.000000,0.000948,0.002677,0.003378,0.164210,0.001423,0.001899,0.000670,0.003197,...,0.000474,0.000967,0.001341,0.003463,0.001655,0.000474,0.002531,0.000987,0.001209,0.001060
acat_inhibitor,0.000921,0.000948,1.000000,0.003092,0.003901,0.001911,0.001643,0.002193,0.000774,0.003692,...,0.000547,0.001117,0.001549,0.003999,0.001911,0.000547,0.002923,0.001139,0.001396,0.001224
acetylcholine_receptor_agonist,0.002602,0.002677,0.003092,1.000000,0.011019,0.005398,0.004641,0.006194,0.002186,0.010429,...,0.001545,0.003156,0.004375,0.011294,0.005398,0.001545,0.008256,0.003218,0.003943,0.003457
acetylcholine_receptor_antagonist,0.003283,0.003378,0.003901,0.011019,1.000000,0.006812,0.005856,0.007816,0.002758,0.013160,...,0.001950,0.003982,0.005521,0.014252,0.006812,0.001950,0.010418,0.004061,0.004975,0.004363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ubiquitin_specific_protease_inhibitor,0.000460,0.000474,0.000547,0.001545,0.001950,0.000955,0.000821,0.001096,0.000387,0.001845,...,0.000273,0.000558,0.000774,0.001999,0.000955,1.000000,0.001461,0.000569,0.000698,0.000612
vegfr_inhibitor,0.002460,0.002531,0.002923,0.008256,0.010418,0.005104,0.004388,0.005856,0.002066,0.009860,...,0.001461,0.002984,0.004136,0.010679,0.005104,0.001461,1.000000,0.003043,0.003728,0.003269
vitamin_b,0.000959,0.000987,0.001139,0.003218,0.004061,0.001989,0.001710,0.002283,0.000805,0.003843,...,0.000569,0.001163,0.001612,0.004162,0.001989,0.000569,0.003043,1.000000,0.001453,0.001274
vitamin_d_receptor_agonist,0.001175,0.001209,0.001396,0.003943,0.004975,0.002437,0.002095,0.002796,0.000987,0.004709,...,0.000698,0.001425,0.001975,0.005099,0.002437,0.000698,0.003728,0.001453,1.000000,0.001561


In [5]:
p_min = 0.001
p_max = 1 - p_min

check_svm2 = check_svm.copy()
check_svm2.loc[cons_train_index,:]= np.clip(check_svm2.loc[cons_train_index,:], p_min, p_max)

check_tabnet2 = check_tabnet.copy()
check_tabnet2.loc[cons_train_index,:] = np.clip(check_tabnet.loc[cons_train_index,:], p_min, p_max)

# output correlation check

In [6]:
#print(mlp.corrwith(xgb).max(), mlp.corrwith(xgb).min(), mlp.corrwith(xgb).mean(), mlp.corrwith(xgb).std())
#print(mlp.corrwith(tab).max(), mlp.corrwith(tab).min(), mlp.corrwith(tab).mean(), mlp.corrwith(tab).std()) 
#print(xgb.corrwith(tab).max(), xgb.corrwith(tab).min(), xgb.corrwith(tab).mean(), xgb.corrwith(tab).std())
#print(mlp.corrwith(tl).max(), mlp.corrwith(tl).min(), mlp.corrwith(tl).mean(), mlp.corrwith(tl).std())
#print(tl.corrwith(tab).max(), tl.corrwith(tab).min(), tl.corrwith(tab).mean(), tl.corrwith(tab).std()) 
#print(xgb.corrwith(tl).max(), xgb.corrwith(tl).min(), xgb.corrwith(tl).mean(), xgb.corrwith(tl).std())

In [7]:
#plt.figure(figsize=(15,30))
#plt.subplot(1,3,1)
#mlp.corrwith(xgb).plot(kind="barh")
#plt.subplot(1,3,2)
##plt.axis('off')
#mlp.corrwith(tab).plot(kind="barh",sharey=True)
#plt.subplot(1,3,3)
##plt.axis('off')
#xgb.corrwith(tab).plot(kind="barh",sharey=True)

In [8]:
check_tabnet2 = pd.DataFrame(np.array(check_tabnet2))
check_tabnet2

,0,1,2,3,4,5,6,7,8,9,...,196,197,198,199,200,201,202,203,204,205
0,0.001544,0.001619,0.001779,0.019419,0.023210,0.006269,0.003881,0.003680,0.001,0.009400,...,0.001000,0.001028,0.002714,0.001620,0.001814,0.001000,0.001000,0.002066,0.001000,0.001665
1,0.001000,0.001000,0.001670,0.036060,0.017666,0.006979,0.004837,0.005461,0.001,0.010227,...,0.001224,0.001166,0.003310,0.001131,0.001693,0.001000,0.001000,0.003152,0.001968,0.003431
2,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001824,0.001,0.001496,...,0.001000,0.001000,0.001000,0.001000,0.023012,0.001000,0.460673,0.001000,0.001000,0.002100
3,0.001000,0.001000,0.001417,0.002797,0.006573,0.001709,0.001000,0.002387,0.001,0.001000,...,0.001000,0.004403,0.001241,0.056919,0.004226,0.001000,0.002674,0.001000,0.001000,0.001000
4,0.001619,0.001756,0.001798,0.011229,0.016726,0.005905,0.005037,0.002912,0.001,0.011807,...,0.001000,0.001000,0.002982,0.001559,0.001442,0.001000,0.001153,0.001647,0.001000,0.001700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,0.001571,0.001665,0.001634,0.013063,0.020870,0.006234,0.004043,0.002976,0.001,0.010642,...,0.001000,0.001000,0.002601,0.001257,0.001755,0.001000,0.001204,0.001793,0.001000,0.001723
23810,0.001000,0.001000,0.004227,0.008601,0.001000,0.001599,0.001186,0.014456,0.001,0.001000,...,0.001000,0.001008,0.006747,0.001000,0.001000,0.001000,0.001000,0.001000,0.053925,0.005557
23811,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
23812,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001,0.001000,...,0.001000,0.001000,0.001000,0.001000,0.002742,0.001000,0.001000,0.001000,0.002581,0.001000


In [9]:
check_dae

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,0.001834,0.001256,0.001144,0.009634,0.034268,0.006291,0.003385,0.004117,0.000616,0.006177,...,0.000695,0.000850,0.001840,0.001644,0.002374,0.000739,0.001633,0.003153,0.000922,0.002719
1,0.000787,0.000791,0.002071,0.015172,0.004746,0.004186,0.005867,0.006436,0.002368,0.010785,...,0.001123,0.001343,0.003038,0.005115,0.003333,0.000559,0.002355,0.002255,0.002829,0.003806
2,0.001307,0.001934,0.001059,0.002058,0.015499,0.005282,0.001235,0.007747,0.000451,0.003523,...,0.000556,0.001923,0.002370,0.002443,0.002510,0.001170,0.022943,0.001254,0.000892,0.001085
3,0.000606,0.000926,0.001818,0.005533,0.006400,0.001925,0.002142,0.001724,0.001032,0.003391,...,0.000874,0.003853,0.002713,0.094964,0.005450,0.000908,0.003084,0.001204,0.000944,0.000963
4,0.001131,0.001156,0.003894,0.009248,0.014102,0.002328,0.005321,0.006814,0.001191,0.008337,...,0.001372,0.003245,0.003876,0.002709,0.003756,0.001184,0.002447,0.002960,0.001864,0.002463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,0.002282,0.001185,0.001988,0.004742,0.019162,0.002475,0.002869,0.004999,0.000645,0.009318,...,0.000737,0.000835,0.002643,0.001313,0.002515,0.000877,0.004641,0.002034,0.001042,0.002265
23810,0.000897,0.001284,0.002493,0.002261,0.003376,0.002828,0.001375,0.019457,0.001189,0.003537,...,0.000692,0.002684,0.002787,0.001736,0.027474,0.001139,0.028338,0.001722,0.006813,0.005605
23811,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
23812,0.000715,0.000650,0.000996,0.001183,0.001950,0.001063,0.001263,0.001822,0.000621,0.003226,...,0.000343,0.000730,0.001265,0.001801,0.002847,0.000342,0.002745,0.000917,0.001401,0.000918


In [10]:
print("mlp dae")
result = []
for i in range(206):
    result.append(np.corrcoef(check_mlp.iloc[:,i], check_dae.iloc[:,i])[0,1])
print(np.max(result), np.min(result), np.mean(result), np.std(result))
print("mlp tab")
result = []
for i in range(206):
    result.append(np.corrcoef(check_mlp.iloc[:,i], check_tabnet.iloc[:,i])[0,1])
print(np.max(result), np.min(result), np.mean(result), np.std(result))

#print("mlp lstm")
#result = []
#for i in range(206):
#    result.append(np.corrcoef(check_mlp.iloc[:,i], check_lstm.iloc[:,i])[0,1])
#print(np.max(result), np.min(result), np.mean(result), np.std(result))

print("mlp svm")
print(check_mlp.corrwith(check_svm).max(), check_mlp.corrwith(check_svm).min(), 
      check_mlp.corrwith(check_svm).mean(), check_mlp.corrwith(check_svm).std())
print("tab dae")
result = []
for i in range(206):
    result.append(np.corrcoef(check_tabnet.iloc[:,i], check_dae.iloc[:,i])[0,1])
print(np.max(result), np.min(result), np.mean(result), np.std(result))

mlp dae
0.9977222337424861 0.3017425383135503 0.6475011401613113 0.13351425561471866
mlp tab
0.9959539601157769 0.0839028380494226 0.5295034711801632 0.17906306979966444
mlp svm
0.9783321950339905 0.02394739236615096 0.46129340145794545 0.2274335068672173
tab dae
0.9968978127323951 0.06142067138276279 0.536993220538671 0.17083265139977447


# ensemble ratio

In [11]:
print('OOF log loss: ', log_loss(np.ravel(targets), np.ravel(check_mlp)))
print('OOF log loss: ', log_loss(np.ravel(targets), np.ravel(check_tabnet)))
print('OOF log loss: ', log_loss(np.ravel(targets), np.ravel(check_svm)))
print('OOF log loss: ', log_loss(np.ravel(targets), np.ravel(check_lstm)))

OOF log loss:  0.01567181329219744
OOF log loss:  0.015617277282027095
OOF log loss:  0.01541462668810311


NameError: name 'check_lstm' is not defined

# simple average

In [12]:
check = 0.5 * check_tabnet.values + 0.5 * check_mlp
print('OOF log loss: ', log_loss(np.ravel(targets), np.ravel(check)))

OOF log loss:  0.01531240108467096


In [13]:
check = 0.3 * check_svm + 0.2 * check_tabnet.values + 0.5 * check_mlp
print('OOF log loss: ', log_loss(np.ravel(targets), np.ravel(check)))

OOF log loss:  0.015048183116221308


# weight average

In [14]:
check = 0.2 * check_tabnet2.values + 0.8 * check_mlp
print('OOF log loss: ', log_loss(np.ravel(targets), np.ravel(check)))

OOF log loss:  0.015456944604645624


In [15]:
check = 0.1 * check_svm2.values + 0.2 * check_tabnet2.values + 0.7 * check_mlp
print('OOF log loss: ', log_loss(np.ravel(targets), np.ravel(check)))

OOF log loss:  0.0152657065477146


In [16]:
check_score = pd.read_csv('../input/lish-moa/train_targets_scored.csv').drop("sig_id", axis=1)
target_feats = [ i for i in check_score.columns if i != "sig_id"]
from scipy.optimize import minimize, fsolve


blend_train = []

# out of fold prediction
blend_train.append(np.array(check_tabnet))
blend_train.append(np.array(check_mlp))
blend_train = np.array(blend_train)
check_xgb2 = np.array(check_xgb2)

ratio = 0.01

np.random.seed(224)
total_scores = []
for i in range(len(target_feats)):
    def log_loss_func(weights):
        final_prediction = ratio * check_xgb2[:,i]
        for weight, prediction in zip(weights, blend_train):
            final_prediction += weight * prediction[:,i]
        return log_loss(np.ravel(targets.iloc[:,i]), np.ravel(final_prediction))
    
    best_score = np.inf
    best_weights = [0] * len(blend_train)
    for k in range(10):
        starting_values = np.random.rand(len(blend_train))
        starting_values /= sum(starting_values)
        bounds = [(0, 1)] * len(blend_train)
        cons = ({'type': 'eq', 'fun': lambda x:  1 - ratio - sum(x)}) 
            
        res = minimize(log_loss_func,
                   starting_values,
                   method='SLSQP',
                   bounds=bounds,
                   constraints = cons) 
        
        if best_score > res["fun"]:
            best_score = res["fun"]
            best_weights = res["x"]
        
    valid_prediction = ratio * check_xgb2[:,i]
    for weight, prediction in zip(best_weights, blend_train):
        valid_prediction += weight * prediction[:,i]
    print('Score: {}, Weights: {}'.format(round(res['fun'],8), res['x']))
    total_scores.append(res['fun'])
    check_score.loc[:,target_feats[i]] = valid_prediction
    
    #oof_test = svm_ratio * svm1_test[:,i]
    #for weight, prediction in zip(best_weights, blend_test):
    #    oof_test += weight * prediction[:,i]
        

print("final ensemble oof score:", np.mean(total_scores))
print('OOF log loss: ', log_loss(np.ravel(targets), np.ravel(check_score)))

NameError: name 'check_xgb2' is not defined

# geometric average

In [17]:
check = np.sqrt(np.multiply(check_tabnet2.values, check_mlp))
print('OOF log loss: ', log_loss(np.ravel(targets), np.ravel(check)))

OOF log loss:  0.015448147951761505


# harmonic average

In [18]:
check_tmp = 1/((1 / check_mlp.loc[cons_train_index,:] + 1 / check_tabnet2.loc[cons_train_index,:].values)/2)
check = check_mlp.copy()
check.loc[cons_train_index,:] = check_tmp
print('OOF log loss: ', log_loss(np.ravel(targets), np.ravel(check)))

OOF log loss:  0.01557513302231724


# n multiply average

In [19]:
num = 4
check = ((check_tabnet_val2.values ** num + check_mlp ** num)/2) ** (1/num)
print('OOF log loss: ', log_loss(np.ravel(targets), np.ravel(check)))

NameError: name 'check_tabnet_val2' is not defined

# lazy ensemble

In [20]:
target_feats = [i for i in targets.columns if i != "sig_id"]
check_mlp.columns = target_feats
check_lstm.columns = target_feats
check_tabnet.columns = target_feats
check_svm.columns = target_feats

NameError: name 'check_lstm' is not defined

In [21]:
# credit: https://www.kaggle.com/reppic/lazy-ensembling-algorithm

import os
# Controls weights when combining predictions
# 0: equal average of all inputs; 
# 1: up to 50% of weight going to least correlated input
DENSITY_COEFF = 0.1
assert DENSITY_COEFF >= 0.0 and DENSITY_COEFF <= 1.0

# When merging 2 files with corr > OVER_CORR_CUTOFF 
# the result's density is the max instead of the sum of the merged files' densities
OVER_CORR_CUTOFF = 0.98
assert OVER_CORR_CUTOFF >= 0.0 and OVER_CORR_CUTOFF <= 1.0

def get_corr_mat(col,frames):
    c = pd.DataFrame()
    for name,df in frames.items():
        c[name] = df[col]
    cor = c.corr()
    for name in cor.columns:
        cor.loc[name][name]=0.0
    return cor

def highest_corr(mat):
    n_cor = np.array(mat.values)
    corr = np.max(n_cor)
    idx = np.unravel_index(np.argmax(n_cor, axis=None), n_cor.shape)
    f1 = mat.columns[idx[0]]
    f2 = mat.columns[idx[1]]
    return corr,f1,f2

def get_merge_weights(m1,m2,densities):
    d1 = densities[m1]
    d2 = densities[m2]
    d_tot = d1 + d2
    weights1 = 0.5*DENSITY_COEFF + (d1/d_tot)*(1-DENSITY_COEFF)
    weights2 = 0.5*DENSITY_COEFF + (d2/d_tot)*(1-DENSITY_COEFF)
    return weights1, weights2

def ensemble_col(col, frames, test_frames, densities):
    if len(frames) == 1:
        _, fr = frames.popitem()
        return fr[col].values

    mat = get_corr_mat(col,frames)
    corr,merge1,merge2 = highest_corr(mat)
    new_col_name = merge1 + '_' + merge2

    w1,w2 = get_merge_weights(merge1,merge2,densities)
    new_df = pd.DataFrame()
    new_df[col] = (frames[merge1][col]*w1) + (frames[merge2][col]*w2)
    del frames[merge1]
    del frames[merge2]
    frames[new_col_name] = new_df

    if corr >= OVER_CORR_CUTOFF:
        #print('\t',merge1,merge2,'  (OVER CORR)')
        densities[new_col_name] = max(densities[merge1],densities[merge2])
    else:
        #print('\t',merge1,merge2)
        densities[new_col_name] = densities[merge1] + densities[merge2]
    del densities[merge1]
    del densities[merge2]
    return ensemble_col(col,frames,test_frames,densities)

ens_check = pd.read_csv('../input/lish-moa/train_targets_scored.csv').drop("sig_id", axis=1)

for col in target_feats:
    print(col)
    oof_dict = {
                'Model 2': check_lstm,
                'Model 3': check_tabnet, 
                'Model 4': check_svm
           }
    
    test_dict = {
     #           'Model 2': lstm1_test,
     #           'Model 3': tabnet1_test, 
     #           'Model 4': svm1_test
           }
    densities = {k:1.0 for k in oof_dict.keys()}
    ens_check[col] = ensemble_col(col,oof_dict,test_dict, densities)

print('OOF log loss: ', log_loss(np.ravel(targets), np.ravel(ens_check)))

5-alpha_reductase_inhibitor


NameError: name 'check_lstm' is not defined

In [22]:
print('OOF log loss: ', log_loss(np.ravel(targets), np.ravel(0.5 * ens_check + 0.5 * check_mlp)))

OOF log loss:  0.003292332675845475
